In [1]:
#Modules to be imported 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from itertools import combinations 
from random import seed
from random import randint
import random

In [4]:
#ANN 

def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_k(x,k):
    return 1 / (1 + k*np.exp(-z))

def relu(x):
    return np.log(1+np.exp(x))

def drelu(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def loss(out,Y):
        loss = (-1)*(np.sum(np.multiply(np.log(out), Y) + np.multiply((1 - Y), np.log(1 - out))))/(Y.shape[1])
        #print(loss)
        return loss

In [5]:
def ann(X,Y,learning_rate):
    #taking all 9 layers as input
    in_layer_no=X.shape[0] #no of attributes
    hid_layer_no=int(2/3*in_layer_no)
    out_layer_no=1

    #initial weights
    wh = np.random.randn(hid_layer_no,in_layer_no) * 0.01
    bh = np.zeros(shape=(hid_layer_no, 1))
    w_out = np.random.randn(out_layer_no,hid_layer_no) * 0.01
    b_out = np.zeros(shape=(out_layer_no, 1))

    initial_weights=[wh,bh,w_out,b_out]
    dwh_old=0
    dw_out_old=0
    for i in range(0,75000):
        #forward propogation
        #input to hidden layer = dot product(X,wh) + bh
        hid_layer_input = np.dot(wh,X) + bh
        hid_layer_act = relu(hid_layer_input)
        
        # Final output layer prediction
        out_layer_input = np.dot(w_out,hid_layer_act) + b_out
        out_layer_act = sigmoid(out_layer_input)
        lo=loss(out_layer_act,Y)
        if(i%1000==0):
            print(i,lo)
        '''dZ2 = out_layer_act - Y
        dW2 = (1 /X.shape[1]) * np.dot(dZ2, hid_layer_act.T)
        db2 = (1 / X.shape[1]) * np.sum(dZ2, axis=1, keepdims=True)
        dZ1 = np.multiply(np.dot(w_out.T, dZ2), 1 - np.power(hid_layer_act, 2))
        dW1 = (1 / X.shape[1]) * np.dot(dZ1, X.T)
        db1 = (1 / X.shape[1]) * np.sum(dZ1, axis=1, keepdims=True)
        
        wh = wh - learning_rate * dW1
        w_out = w_out - learning_rate * dW2
        bh = bh - learning_rate * db1
        b_out = b_out - learning_rate * db2
        '''
        #backward propogation output layer    
        t_o = out_layer_act - Y
        sigmak = t_o * sigmoid(out_layer_input)*(1-sigmoid(out_layer_input))   
        dLoss_W2 = (1/hid_layer_act.shape[1]) * np.dot(sigmak,hid_layer_act.T)
        dLoss_b2 = (1/hid_layer_act.shape[1]) * np.dot(sigmak, np.ones([sigmak.shape[1],1])) 
          
        #backward propogation input layer
        dLoss_A1 = np.dot(w_out.T,sigmak)
        dLoss_Z1 = dLoss_A1 * drelu(hid_layer_input)        
        dLoss_A0 = np.dot(wh.T,dLoss_Z1)
        dLoss_W1 = 1/X.shape[1] * np.dot(dLoss_Z1,X.T)
        dLoss_b1 = 1/X.shape[1] * np.dot(dLoss_Z1, np.ones([dLoss_Z1.shape[1],1]))  
        
        wh = wh - learning_rate * dLoss_W1
        w_out = w_out - learning_rate * dLoss_W2
        bh = bh - learning_rate * dLoss_b1
        b_out = b_out - learning_rate * dLoss_b2
    return [wh,bh,w_out,b_out]
    ''' 
        #backpropogation
        dout_layer_act = (out_layer_act - Y) / (out_layer_act * (1 - out_layer_act))
        dZ2 = np.multiply(dout_layer_act, out_layer_act * (1 - out_layer_act))
        dw_out = np.dot(dZ2, hid_layer_act.T)
        #dw_out=dw_out+0.5*dw_out_old
        #dw_out_old=dw_out
        db_out = np.sum(dZ2, axis=1, keepdims=True)

        dhid_layer_act = np.dot(w_out.T, dZ2)
        dZ1 = np.multiply(dhid_layer_act, dhid_layer_act * (1 - dhid_layer_act))
        dwh = np.dot(dZ1, X.T)
        #dwh=dwh+0.2*dwh_old
        #dwh_old=dwh
        dbh = np.sum(dZ1, axis=1, keepdims=True)
        
        wh = wh - learning_rate * dwh
        w_out = w_out - learning_rate * dw_out
        bh = bh - learning_rate * dbh
        b_out = b_out - learning_rate * db_out
    '''
        
        

In [6]:
#final forward propogation to test trained model 
def predict(XTrain,YTrain,XTest,YTest,learning_rate):
    final_weights=ann(XTrain,YTrain,learning_rate)
    hid_layer_input = np.dot(final_weights[0],XTest) + final_weights[1]
    hid_layer_act = relu(hid_layer_input)
    out_layer_input = np.dot(final_weights[2],hid_layer_act) + final_weights[3]
    out_layer_act = sigmoid(out_layer_input)
    predictions = out_layer_act > 0.5
    predictions=predictions.astype(int)  
    accuracy=float((np.dot(YTest,predictions.T) + np.dot(1-YTest,1-predictions.T))/float(YTest.size)*100)
    #print(accuracy)
    #print ('Accuracy: %f' % float((np.dot(YTest,predictions.T) + np.dot(1-YTest,1-predictions.T))/float(YTest.size)*100) + '%')
    return accuracy

In [7]:
def split_test_train(X,Y):
    #print(X.shape,Y.shape)
    XTrain,XTest,YTrain,YTest=train_test_split(X, Y, test_size=0.20, random_state=42)
    X,Y=X.to_numpy(),Y.to_numpy()
    XTrain,XTest,YTrain,YTest= XTrain.to_numpy().T,XTest.to_numpy().T,YTrain.to_numpy().reshape(1, YTrain.shape[0]),YTest.to_numpy().reshape(1, YTest.shape[0])
    X,Y=X.T,Y.reshape(1, Y.shape[0])
    #print(X.shape,Y.shape,XTrain.shape,YTrain.shape,XTest.shape,YTest.shape)
    acc=predict(XTrain,YTrain,XTest,YTest,0.05)
    return(acc)


In [9]:
#Function to create all possible combination of subsets of size r
def rSubset(arr,r): 
    return list(combinations(arr, r)) 
def comb(r):
    arr = [2,3,4,5,6,7,8,9,10] 
    return(rSubset(arr, r))

#function for bagging of attributes 
def attribute_bagging(r):
    s=comb(r)
    #print(s,len(s))
    length=len(s)
    avg=[]
    #repeat 5 times
    for i in range(0,5):
        #9 random values
        #generating 9 random values with range given as the 0,number of combination
        rand_index=random.sample(range(0, length), 9)
        l=[]
        for j in rand_index:
            t=s[j]
            #subsetting the dataframe X based on rand_index values
            new_dataset = X.loc[:,list(t)]
            #sending the new dataframe to the ann function 
            acc=split_test_train(new_dataset,Y)
            #saving accuracy of each accuracy in a list
            l.append(acc)
        #finding average of accuracies of each iteration
        avg1=sum(l)/len(l) 
        avg.append(avg1)
    return(avg)
            
    
    
def ANN_att_bag():
    total=[]
    #attribute bagging for subset size 2,3,4,5,6,7,8 and 9 each time
    for i in range(2,10):
        #append accuracies of each subset size
        total.append(attribute_bagging(i))
    max_avg=[]
    #taking max of accuracies for each subset size 
    for i in total:
        max_avg.append(max(i))
    #print(max_avg)
    #plot to show accuracies for each subset 
    keys=[2,3,4,5,6,7,8,9]
    #converting to a dictionary
    dictionary = dict(zip(keys, max_avg))
    print(dictionary)
    #p
    plt.plot(*zip(*sorted(dictionary.items())))
    plt.show()


In [ ]:
#RANDOM FOREST

In [ ]:
if __name__ == "__main__":
    #reading preprocessed dataset
    df_proc = pd.read_csv("preprocessed.csv") 
    df_proc.columns=[0,1,2,3,4,5,6,7,8,9,10,11]
    #finding X(attributes excluding Target) and Y(Target)
    Y = df_proc.filter([11], axis=1)
    X = df_proc.drop([0,1,11],axis=1)
    #ANN 
    ANN_att_bag()
    #RANDOM FOREST
    
    